In [41]:
from parsimonious.grammar import Grammar
from parsimonious.nodes import NodeVisitor
from abc import ABC
import numpy as np 
from NameCreator import NameCreator
import networkx as nx
import matplotlib.pyplot as plt 
from tree import *

%matplotlib inline
grammar = Grammar(
     """
        expr = (term add expr) /  term
        term = (factor mul term) /  factor
        factor = (left expr right) / const 
        add = "+"
        mul = "*"
        left = "("
        right = ")"
        const = ~"[A-Z 0-9]*"i
     """)
tree = grammar.parse("a+b+c")

In [42]:
def parse_expr(text,lookup):
    tree = grammar.parse(text)
    iv = IniVisitor()
    output = iv.visit(tree)
    return create_binary_tree(output,lookup)

In [43]:
l = [("a","1"),("b","2"),("c","a+b")]
lookup = dict()
for name,value in l:
    try:
        v = float(value)  
        tensor = Variable(name,float(value))
        lookup[name] = tensor
    except ValueError:
        tensor = parse_expr(value,lookup)
        tensor.name = name
        lookup[name] = tensor

In [44]:
class IniVisitor(NodeVisitor):
    def visit_expr(self, node, visited_children):
        """ Returns the overall output. """
        output = []
        for child in visited_children[0]:
            output.append(child)
        return output
    
    def visit_term(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return visited_children[0]

    def visit_factor(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return visited_children[0]
    
    def visit_const(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return node.text
    def visit_add(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return node.text
    def visit_mul(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return node.text

    def visit_left(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return node.text
    
    def visit_right(self, node, visited_children):
        """ Gets each key/value pair, returns a tuple. """
        return node.text

    def generic_visit(self, node, visited_children):
        """ The generic visit method. """
        return visited_children or node

In [45]:
iv = IniVisitor()
output = iv.visit(tree)
print(output)

['a', '+', ['b', '+', ['c']]]


In [46]:
class Tensor(ABC):
    def calculation(self):
        #Compute the value of current node
        pass

    def accept(self,visitor,ctx):
        visitor.visit(self,ctx)

class Expression(Tensor):
    def __init__(self,lhs:Tensor,op:str,rhs:Tensor,name=None):
        self.lhs = lhs
        self.rhs = rhs
        self.op = op
        self.value = self.calculation()
        self.grad = 0
        self.name = name if name else NameCreator.getName()
        
    def calculation(self):
        if self.op == '+':
            return self.lhs.calculation() + self.rhs.calculation()
        else:
            return self.lhs.calculation() * self.rhs.calculation()

    def accept(self,visitor,ctx):
        visitor.visit_Expression(self,ctx)

    def __str__(self,level=0):
        return self.get_str()

    def get_str(self,level=0):
        return '\n'+'\t'*level+"<var: "+self.name+ " grad:"+str(self.grad)+" value:"+str(self.value)+">"+\
                self.lhs.get_str(level+1)+\
                '\n'+'\t'*(level+1)+ '<'+str(self.op)+'/>'+\
                self.rhs.get_str(level+1)+\
                '\n'+'\t'*level+"</>"
    
class Variable(Tensor):
    def __init__(self,name:str,value:float=0):
        self.name = name
        self.value = np.random.randint(1,10,1)[0]
        self.grad = 0
        
    def calculation(self):
        return self.value
    
    def accept(self,visitor,ctx):
        visitor.visit_Variable(self,ctx)
    
    def get_str(self,level=0):
        return '\n'+'\t'*level+"<"+'var:'+ self.name + " value:"+str(self.value)+ ' grad:' +str(self.grad)+"/>"
    
    def __str__(self,level=0):
        return self.get_str()

def create_binary_tree(output,lookup):
    """
    :param output: output of visitor
    :param lookup: dictionary map name -> Variable object
    """
    if type(output) is str:
        if output in lookup:
            instance = lookup[output]
            if not isinstance(instance,Expression):
                return instance
            return Expression(create_binary_tree(instance.lhs.name,lookup),
                        instance.op,
                        create_binary_tree(instance.rhs.name,lookup),output)
        else:
            tmp = Variable(output)
            lookup[output] = tmp
            return tmp

    # 
    if len(output) == 1:
        if output[0] in lookup:
            instance = lookup[output[0]]
            if not isinstance(instance,Expression):
                return instance
            return Expression(create_binary_tree(instance.lhs.name,lookup),
                        instance.op,
                        create_binary_tree(instance.rhs.name,lookup),output[0])
        else:
            tmp = Variable(output[0])
            lookup[output[0]] = tmp
            return tmp
    else:
        # parenthesis => calculate the expression inside it
        if output[0] == '(':
            return create_binary_tree(output[1],lookup)
        # recursive construct tree
        return Expression(create_binary_tree(output[0],lookup),
                        output[1],
                        create_binary_tree(output[2],lookup))

In [47]:
tree = create_binary_tree(output,lookup)

In [48]:
print(str(tree))


<var: @temp_8 grad:0 value:30>
	<var:a value:9 grad:0/>
	<+/>
	<var: @temp_7 grad:0 value:21>
		<var:b value:6 grad:0/>
		<+/>
		<var: c grad:0 value:15>
			<var:a value:9 grad:0/>
			<+/>
			<var:b value:6 grad:0/>
		</>
	</>
</>


In [49]:
class BackwardVisitor():
    def visit(self,node,ctx):
        node.accept(self,ctx)

    def calc_grad(self,node,ctx):
        op = ctx['op']
        prev_grad = ctx['grad']
        co_value = ctx['value']

        if op == '+':
            node.grad += prev_grad
        else:
            node.grad += prev_grad*co_value

    def visit_Expression(self,node,ctx):
        self.calc_grad(node,ctx)
        ctx_lhs = {'grad':node.grad,'op':node.op,'value':node.rhs.value}
        ctx_rhs = {'grad':node.grad,'op':node.op,'value':node.lhs.value}
        self.visit(node.lhs,ctx_lhs)
        self.visit(node.rhs,ctx_rhs)
    
    def visit_Variable(self,node,ctx):
        self.calc_grad(node,ctx)

In [50]:
v = BackwardVisitor()
ctx = {'grad':1,'op':'+','value':0}
v.visit(tree,ctx)

In [51]:
print(str(tree))


<var: @temp_8 grad:1 value:30>
	<var:a value:9 grad:2/>
	<+/>
	<var: @temp_7 grad:1 value:21>
		<var:b value:6 grad:2/>
		<+/>
		<var: c grad:1 value:15>
			<var:a value:9 grad:2/>
			<+/>
			<var:b value:6 grad:2/>
		</>
	</>
</>


In [52]:
def find_edges(head):
    ret = []
    if isinstance(head,Variable):
        return ret
    else:
        left_edges = find_edges(head.lhs)
        right_edges = find_edges(head.rhs)
        ret = ret + left_edges + right_edges
        ret.append((head.lhs.name,head.name,head.lhs_grad))
        ret.append((head.rhs.name,head.name,head.rhs_grad))
        return ret

def find_nodes(head):
    ret = dict()
    if isinstance(head,Variable):
        ret[head.name] = 'var'
        return ret
    else:
        left_nodes = find_nodes(head.lhs)
        right_nodes = find_nodes(head.rhs)
        ret.update(left_nodes)
        ret.update(right_nodes)
        ret[head.name] = 'add' if head.op == '+' else 'mul'
        return ret

In [53]:
v = find_nodes(tree)
e = find_edges(tree)

colormap = list()
for k in v:
    if v[k] == 'var':
        colormap.append('#3F51B5')
    elif v[k] == 'add':
        colormap.append('#FFC107')
    else:
        colormap.append('#F44336')
v = list(v)
#colormap = np.arange(len(colormap))/10
e.reverse()
G = nx.DiGraph()

G.add_nodes_from(v)
for edge in e:
    src, tgt, w = edge
    G.add_edge(tgt, src, weight=w)

edge_labels = nx.get_edge_attributes(G, 'weight')
norm = plt.Normalize(1, 4)
cmap = plt.cm.RdYlGn
fig, ax = plt.subplots()
plt.title('draw_networkx')
pos = nx.drawing.nx_agraph.graphviz_layout(
    G, prog='dot', args='-Nfontsize=10 -Nwidth="10" -Nheight="1" -Nmargin=0 -Gfontsize=8')
nx.draw(G, pos, with_labels=True, arrows=True,
        node_size=1500, node_color=colormap)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
plt.show()

AttributeError: 'Variable' object has no attribute 'lhs'

In [54]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig = plt.figure()

G = nx.Graph()
G.add_edges_from(e)
G.add_nodes_from(v)

pos=nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')
edges = nx.draw_networkx_edges(G, pos, edge_color = 'w',with_labels=True)
nodes = nx.draw_networkx_nodes(G, pos, node_color = 'g',with_labels=True)

white = (1,1,1,1)
black = (0,0,0,1)

colors = [white for edge in edges.get_segments()]

def update(n):
    global colors

    try:
        idx = colors.index(white)
        colors[idx] = black
    except ValueError:
        colors = [white for edge in edges.get_segments()]

    edges.set_color(colors)
    return edges, nodes

anim = FuncAnimation(fig, update, interval=150, blit = True) 

plt.show()

NameError: name 'e' is not defined

<Figure size 432x288 with 0 Axes>

In [30]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

e.reverse()
G = nx.DiGraph()

G.add_nodes_from(v)
G.add_edges_from(e)

plt.title('draw_networkx')
pos=nx.drawing.nx_agraph.graphviz_layout(G, prog='dot',args='-Nfontsize=10 -Nwidth="10" -Nheight="1" -Nmargin=0 -Gfontsize=8')
nx.draw(G, pos, with_labels=True, arrows=True,node_size=1000,node_color=colormap)
plt.show()

NameError: name 'e' is not defined

In [1]:
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(1)
x = np.random.rand(15)
y = np.random.rand(15)
names = np.array(list("ABCDEFGHIJKLMNO"))
c = np.random.randint(1,5,size=15)
norm = plt.Normalize(1,4)
cmap = plt.cm.RdYlGn
fig,ax = plt.subplots()
sc = plt.scatter(x,y,c=c, s=100, cmap=cmap, norm=norm)
annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)
def update_annot(ind):
    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
                           " ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
    annot.get_bbox_patch().set_alpha(0.4)
def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
fig.canvas.mpl_connect("motion_notify_event", hover)
plt.show()

<Figure size 640x480 with 1 Axes>